In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sweetviz as sv
import missingno as msno
import time 
import sys
import os
import csv
from bs4 import BeautifulSoup
import requests
import re

from sklearn.compose import make_column_selector, ColumnTransformer, make_column_transformer
# sklearn.compose: The sklearn.compose module is a submodule of the sklearn library for machine learning in Python. It provides functions for creating complex preprocessing and modeling pipelines.
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PolynomialFeatures,RobustScaler
#sklearn.preprocessing: The sklearn.preprocessing module is a submodule of the sklearn library that provides functions for preprocessing data, such as scaling and normalizing features, imputing missing values, and encoding categorical variables.
from sklearn.linear_model import Ridge,LinearRegression,Lasso, ElasticNet
# sklearn.linear_model: The sklearn.linear_model module is a submodule of the sklearn library that provides functions for fitting linear models for regression and classification.
from sklearn.pipeline import make_pipeline
# sklearn.pipeline: The sklearn.pipeline module is a submodule of the sklearn library that provides functions for creating and working with pipelines of transformers and models.
from sklearn.model_selection import train_test_split,GridSearchCV,learning_curve, RandomizedSearchCV, cross_val_score, KFold
# sklearn.model_selection: The sklearn.model_selection module is a submodule of the sklearn library that provides functions for splitting data into training and test sets, evaluating models using cross-validation, and hyperparameter tuning.
from sklearn.dummy import DummyRegressor
# sklearn.dummy: The sklearn.dummy module is a submodule of the sklearn library that provides simple dummy models for regression and classification.


from sklearn.impute import SimpleImputer
import numpy as np

from sklearn.ensemble import RandomForestClassifier
import pickle


In [2]:
try:
    df = pd.read_csv('/home/apprenant/Documents/archive/SBAnational.csv')
except:
    df = pd.read_csv('C:/Users/emada/Downloads/loan_project (1)/SBAnational.csv')

df['ApprovalFY'] = df['ApprovalFY'].replace('A', '', regex=True).astype(int)

/tmp/ipykernel_17736/1292895530.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/apprenant/Documents/archive/SBAnational.csv')


In [3]:
# create a list of all numeric columns
columns_to_transform_to_int = ["DisbursementGross","BalanceGross","ChgOffPrinGr","GrAppv","SBA_Appv"]


for col in columns_to_transform_to_int:
    df[col] = df[col].str.replace("$", "")
    df[col] = df[col].str.replace(",", "")
    df[col] = df[col].astype(float)
    

df['Term'] = df['Term'].astype(int)

df['LowDoc'] = df['LowDoc'].replace({'0': 'N'})
df = df[df['LowDoc'].isin(['N','Y'])]
df['RevLineCr'] = df['RevLineCr'].replace({'0': 'N', 'T':'Y'})
df = df[df['RevLineCr'].isin(['N','Y'])]

dictionnaire =  {"11":"Agriculture, forestry, fishing and hunting","72":"Accommodation and food services","21":"Mining, quarrying, and oil and gas extraction","22":
"Utilities","23":"Construction","31":"Manufacturing","32":"Manufacturing","33":"Manufacturing","42":"Wholesale trade","44":"Retail trade","45":"Retail trade","48":" Transportation and warehousing", "49":"Transportation and warehousing", "51":"Information","52":"Finance and insurance", "53":"Real estate and rental and leasing","54":"Professional, scientific, and technical services","55":"Management of companies and enterprises","56":"Administrative and support and waste management and remediation services","61":"Educational services","62":"Health care and social assistance","71":"Arts, entertainment, and recreation","81":"Other services (except public administration)", "92": "Public administration", "0" : "Other"}

df['NAICS'] = df['NAICS'].astype(str).str[:2]

df["NAICS"] = df["NAICS"].map(dictionnaire)

cols_to_drop = ['LoanNr_ChkDgt', 'Name', 'City', 'Zip', 'Bank', 'BankState', 'ApprovalDate', 'RetainedJob','ChgOffDate', 'DisbursementDate', 'DisbursementGross', 'BalanceGross', 'SBA_Appv', 'ChgOffPrinGr']
df_cleaned = df.copy()
df_cleaned.drop(columns=cols_to_drop, inplace=True)

df_cleaned.dropna(subset=['MIS_Status'], inplace=True)


/tmp/ipykernel_17736/2495417656.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace("$", "")


In [4]:
y = df_cleaned.MIS_Status
X = df_cleaned.drop(columns=['MIS_Status'])

In [5]:
def make_pipeline_to_ML(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.8, random_state=42)
    numerical_features = make_column_selector(dtype_include=np.number)
    categorical_features = make_column_selector(dtype_exclude= np.number)
    numerical_pipeline = make_pipeline(SimpleImputer(strategy='mean'),StandardScaler(with_mean=False))
    categorical_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder(handle_unknown='ignore'))
    preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                                    (categorical_pipeline, categorical_features)
                                    )
    return preprocessor, X_train, X_test, y_train, y_test


# Create the pipeline
preprocessor, X_train, X_test, y_train, y_test = make_pipeline_to_ML(X,y)

# XGBClassifier
!['XGD'](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRD7Jd4XJNHb0wB_i4V7NFA_qrc6LTEuxPhgw&usqp=CAU)

In [6]:
from sklearn.preprocessing import LabelEncoder

# initialize the label encoder
le = LabelEncoder()

# fit and transform the y labels
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)


In [56]:
from xgboost import XGBClassifier
clf_egb_RGCV = make_pipeline(preprocessor, XGBClassifier( random_state=42, objective = 'binary:logistic', tree_method = 'hist'))
params = {
        'xgbclassifier__max_depth':[1,5,10], 
        #'xgbclassifier__objective' : ['binary:logistic'],
        'xgbclassifier__n_estimators' : [100,200],
        'xgbclassifier__learning_rate' : [0.2,0.3],
        #'xgbclassifier__tree_method': ['hist']   # this will make it faaaaaaaaaster
        }
 # Create the grid search object
grid_seargrid_search_xgboostch = GridSearchCV(clf_egb_RGCV, params, cv=5, n_jobs = -1, verbose=2)

In [ ]:
# Fit the grid search to the data
grid_seargrid_search_xgboostch.fit(X_train, y_train)

In [58]:
# Print the best parameters and the best score
print("Best parameters: ", grid_seargrid_search_xgboostch.best_params_)
print("Best score: ", grid_seargrid_search_xgboostch.best_score_)

Best parameters:  {'xgbclassifier__learning_rate': 0.2, 'xgbclassifier__max_depth': 10, 'xgbclassifier__n_estimators': 200}
Best score:  0.9485839437459355


In [59]:
# Get the accuracy score on the test data
accuracy = grid_seargrid_search_xgboostch.score(X_test, y_test)

# Print the accuracy score
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 94.89%


In [47]:
clf_xgb = make_pipeline(preprocessor, XGBClassifier(random_state=42, 
                                                max_depth = 10, 
                                                n_estimators = 200,
                                                #min_child_weight = 4800,
                                                #subsample = 0.8,
                                                #colsample_bytree = 0.8,
                                                #scale_pos_weight=1.375,
                                                objective='binary:logistic',
                                                learning_rate=0.2,
                                                tree_method = 'hist',
                                                n_jobs=-1))

# Fit the pipeline on the training data
clf_xgb.fit(X_train, y_train)


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler(with_mean=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f26eecbc7f0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_freque...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=0.2,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=10, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=200,
                               n_jobs=-1, num_parallel_tree=None,
                               predictor=None, random_state=42, ...))])

In [48]:
from sklearn.model_selection import cross_val_predict
predicted = cross_val_predict(clf_xgb, X_train, y_train, cv=5, n_jobs=-1, verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   36.7s remaining:   55.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   38.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   38.9s finished


In [49]:
from sklearn.metrics import classification_report
print(classification_report(y_train, predicted))

              precision    recall  f1-score   support

           0       0.87      0.83      0.85    124636
           1       0.96      0.97      0.97    585823

    accuracy                           0.95    710459
   macro avg       0.92      0.90      0.91    710459
weighted avg       0.95      0.95      0.95    710459



In [50]:
from sklearn.metrics import f1_score
# Predict on the test data
y_pred = grid_seargrid_search_xgboostch.predict(X_test)

print(f"f1_score :{f1_score(y_test, y_pred, average='weighted')}")

f1_score :0.9483659639200629


In [51]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 25993,   5288],
       [  3795, 142539]])

In [52]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.83      0.85     31281
           1       0.96      0.97      0.97    146334

    accuracy                           0.95    177615
   macro avg       0.92      0.90      0.91    177615
weighted avg       0.95      0.95      0.95    177615



In [53]:
import pickle

# Save the model to a file
with open("XGBoost_model.pkl", "wb") as file:
    pickle.dump(clf_xgb, file)